In [ ]:
import os
import numpy as np
import cv2
import lmdb
import pickle


# Setting the paths for the base directory of images and the directory to save the LMDB
base_imgs_dir = "YOUR PATH/people"
save_dir = "YOUR PATH/LMDB"

# Creating the save_dir directory if it does not exist
if not os.path.exists(save_dir): os.makedirs(save_dir)
# Creating the path for the LMDB directory
lmdb_dir = os.path.join(save_dir, 'lmdb')
# Creating the path for the keys file to store the keys and person IDs
key_path = os.path.join(save_dir, 'keys.pkl')
# Setting the base_dir equal to base_imgs_dir
base_dir = base_imgs_dir
# Creating the lmdb_dir directory if it does not exist
if not os.path.exists(lmdb_dir): os.makedirs(lmdb_dir)

# Initializing the lists to store keys, names of videos, and person IDs
keys = []
vnames = []
pids = []
# Opening an LMDB environment for writing
env = lmdb.open(lmdb_dir, map_size=1e12)
# Starting a write transaction
txn = env.begin(write=True)
# Starting a write transaction
cnt = 0
vid_cnt = 0
# Sorting the countries in the base directory
countries = sorted(os.listdir(base_dir))
# Loop through the countries
for i, country in enumerate(countries):
    # Creating the path for the current city
    city_dir = os.path.join(base_dir, country)
    # Sorting the cities in the current country
    cities = sorted(os.listdir(city_dir))
    # Loop through the cities
    for j, city in enumerate(cities):
        # Creating the path for the current video directory
        vid_dir = os.path.join(city_dir, city)
        # Sorting the videos in the current city
        vids = sorted(os.listdir(vid_dir))
        # Loop through the videos
        for k, vid in enumerate(vids):
            # Skip the video if it has already been processed
            if vid in vnames: continue
            # Adding the current video name to the vnames list
            vnames.append(vid)
            # Incrementing the video counter
            vid_cnt += 1
            # Creating a string for the current video key
            key_vid = '{:05d}'.format(vid_cnt)
            # Creating the path for the current image directory
            im_dir = os.path.join(vid_dir, vid)
            # Sort the names of the images and only select those that end with "jpg"
            names = sorted([x for x in os.listdir(im_dir) if x.endswith('jpg')])
            # Loop through the images
            for m, name in enumerate(names):
                # Print the current image processing status
                if cnt % 2000 == 0:
                    print('[{:3d}|{:3d}] country={:s}, [{:d}|{:d}] city={:s}, ' \
                        '[{:3d}|{:3d}] vid={:s}, [{:d}|{:d}] name={:s}'.format(i,
                            len(countries), country, j, len(cities), city, k, len(vids),
                            vid, m, len(names), name))
                # Create the key for the current image
                key = key_vid + '_' + name[:-4]
                # Extract the person ID from the name of the image
                pid = int(name.split('_')[0])
                # Create the path to the current image
                im_path = os.path.join(im_dir, name)
                # Open the current image
                with open(im_path, 'rb') as f:
                    im_str = f.read()
                im = np.fromstring(im_str, np.uint8)
                # Add the image key, person ID, and image data to the respective lists
                keys.append(key)
                pids.append(pid)
                # Convert the key to bytes
                key_byte = key.encode('ascii')
                # Store the image data in the LMDB transaction
                txn.put(key_byte, im)
                # Increment the count of processed images
                cnt += 1
        # Commit the LMDB transaction
        txn.commit()
        # Start a new LMDB transaction
        txn = env.begin(write=True)
    # Commit the transaction and close the LMDB environment
    txn.commit()
    env.close()

    # Store the lists of keys and person IDs in a pickle file
    with open(key_path, 'wb') as f:
        pickle.dump({"keys": keys, "pids":pids}, f)

<ipython-input-3-acba92c97d65>:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  env = lmdb.open(lmdb_dir, map_size=1e12)


[  0|  1] country=argentina, [0|1] city=buenos_aires, [  1|  3] vid=0aGt5HKP_FI, [0|31] name=000017_0000181_00.jpg


<ipython-input-3-acba92c97d65>:50: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  im = np.fromstring(im_str, np.uint8)
